# Duckietown NCTU - Tutorial 5: Duckietown-Description
By Brian Chuang, Kenny Su, and Nick Wang

Open another terminal to roslaunch a duckietown description, e.g., $ roslaunch duckietown_description duckietown_description_node.launch veh:=wama map_name:=open_house_dp6

## Import Packages

In [1]:
import numpy as np
import scipy as sp
import cv2
import time

from matplotlib import pyplot as plt
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels

## ROS Setup

In [2]:
import sys
# rospy
sys.path.insert(0, '/opt/ros/indigo/lib/python2.7/dist-packages')
# rospkg
sys.path.insert(0, '/usr/lib/python2.7/dist-packages/')

# duckietown_msgs
duckietown_root = '../../'  # this file should be run from {duckietown_root}/turorials/python (otherwise change this line)
sys.path.insert(0, duckietown_root + 'catkin_ws/devel/lib/python2.7/dist-packages')
import roscpp
import rospy
import tf2_ros
import PyKDL
import math
import tf
#from geometry_msgs.msg import Point, Quaternion,Transform
from geometry_msgs.msg import TransformStamped
from tf_conversions import posemath

## Initial a Ros Node

In [ ]:
rospy.init_node("state_duckiebot_publisher",anonymous=False)

## Define Duckiebot Transformation Publisher

In [ ]:
trans_pub = tf2_ros.TransformBroadcaster()
trans_msg = TransformStamped()

def state_duckiebot_publish(transformation, head_id, child_id):
    
    xyz = tf.transformations.translation_from_matrix(transformation)
    quat = tf.transformations.quaternion_from_matrix(transformation)
    trans_msg.header.frame_id = head_id
    trans_msg.child_frame_id = child_id
    trans_msg.transform.translation.x = xyz[0]
    trans_msg.transform.translation.y = xyz[1]
    trans_msg.transform.translation.z = xyz[2]
    trans_msg.transform.rotation.x = quat[0]
    trans_msg.transform.rotation.y = quat[1]
    trans_msg.transform.rotation.z = quat[2]
    trans_msg.transform.rotation.w = quat[3]
    trans_pub.sendTransform(trans_msg) 
    

## Translation, Rotation, Transformation example

* example 1: translation 

In [ ]:
xyz = (1,0,0)

translation = tf.transformations.translation_matrix((xyz))
print 'translation: \n', translation

xyz_ = tf.transformations.translation_from_matrix(translation)
print 'extract xyz from translation: \n', xyz_

state_duckiebot_publish(translation, 'world', 'duckiebot')

* example 2: rotation by rpy

In [ ]:
rpy = (0,0,math.pi)

rotation_rpy = tf.transformations.euler_matrix(rpy[0], rpy[1], rpy[2], 'rxyz')
print 'rotation by rpy: \n', rotation_rpy

rpy_ = tf.transformations.euler_from_matrix(rotation_rpy)
print 'extract rpy from rotation: \n', rpy_

state_duckiebot_publish(rotation_rpy, 'world', 'duckiebot')

* example 3: rotation by quaternion

In [ ]:
quat = (0, 0, 1, 1)

rotation_quat = tf.transformations.quaternion_matrix(quat)
print 'rotation by quat: \n', rotation_quat

quat_ = tf.transformations.quaternion_from_matrix(rotation_quat)
print 'extract quaternion from rotation: \n', quat_

rpy_quat = tf.transformations.euler_from_quaternion(quat_)
print 'convert quaternion: ', quat_, '\n to rpy:', rpy_quat

state_duckiebot_publish(rotation_quat, 'world', 'duckiebot')

* example 4: combine translation and rotation

In [10]:
xyz = (0.6,0,0)
translation = tf.transformations.translation_matrix((xyz))
print 'translation: \n', translation

rpy = (0,0,math.pi)
rotation = tf.transformations.euler_matrix(rpy[0], rpy[1], rpy[2], 'rxyz')
print 'rotation: \n', rotation

T = translation.dot(rotation)
print 'combine translation and rotation: \n', T
state_duckiebot_publish(T, 'world', 'duckiebot')

translation: 
[[ 1.   0.   0.   0.6]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.   1. ]]
rotation: 
[[ -1.00000000e+00  -1.22464680e-16  -0.00000000e+00   0.00000000e+00]
 [  1.22464680e-16  -1.00000000e+00   0.00000000e+00   0.00000000e+00]
 [ -0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
combine translation and rotation: 
[[ -1.00000000e+00  -1.22464680e-16   0.00000000e+00   6.00000000e-01]
 [  1.22464680e-16  -1.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


* define function of geting combination translation and rotation

In [9]:
def get_trans_from_xyz_rpy(xyz, rpy):
    translation = tf.transformations.translation_matrix((xyz))
    rotation = tf.transformations.euler_matrix(rpy[0], rpy[1], rpy[2], 'rxyz')
    T = np.matmul(translation, rotation)
    return T

* example 5: composition - repeat the transformation in example 4 four times 

In [11]:
xyz = (0.6, 0, 0)
rpy = (0, 0, math.pi / 4)
T = get_trans_from_xyz_rpy(xyz, rpy)
T_ = T
for i in range(4):
    print 'transformation times: ', i+1
    state_duckiebot_publish(T_, 'world', 'duckiebot')
    rospy.sleep(3)
    print T_
    T_ = np.matmul(T_, T)
    

transformation times:  1
[[ 0.70710678 -0.70710678  0.          0.6       ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
transformation times:  2
[[  2.22044605e-16  -1.00000000e+00   0.00000000e+00   1.02426407e+00]
 [  1.00000000e+00   2.22044605e-16   0.00000000e+00   4.24264069e-01]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
transformation times:  3
[[-0.70710678 -0.70710678  0.          1.02426407]
 [ 0.70710678 -0.70710678  0.          1.02426407]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
transformation times:  4
[[ -1.00000000e+00  -2.77555756e-16   0.00000000e+00   6.00000000e-01]
 [  2.77555756e-16  -1.00000000e+00   0.00000000e+00   1.44852814e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.000000